#### TODO
- Check how to get around article limit of 10000 if possible
- Can use history server for other purposes?
- What can be the advantage for using eutilities VS edirect / usecases?

In [32]:
from Bio import Entrez
import time

In [33]:
def build_query():
    searchterm = input("Enter query search term: ")
    article_type = ('[IT] AND ("Clinical Trial"[PT] OR "Randomized Controlled Trial"[PT] OR "Meta-Analysis"[PT] '
                    'OR "Systematic Review"[PT] OR "Comparative Study"[PT] OR "Observational Study"[PT] '
                    'OR "Validation Study"[PT] OR "Case Reports"[PT] OR "Review"[PT])')
    
    full_query = searchterm + article_type
    return full_query


In [34]:
def history(full_query):
    API_key = input("Enter API_key:")

    # Get number of data records
    with Entrez.esearch(db="pubmed", term=query, retmax=1, api_key=API_key, usehistory="y") as handle:
        results = Entrez.read(handle)
        count = int(results["Count"])
        webenv = results["WebEnv"]    
        query_key = results["QueryKey"]
    
    print(f"Total articles found: {count}")
    print(f"WebEnv: {webenv}, QueryKey: {query_key}")
    
    return webenv, query_key, count, API_key


In [30]:
def fetch_data(webenv, query_key, count, API_key, batchsize=5000):
    article_data = []

    for start in range(0, count, batchsize):
        print(f"Records {start + 1} to {min(start + batchsize, count)}...")
        
        with Entrez.efetch(db="pubmed", query_key=query_key, webenv=webenv, retstart=start, retmax=batchsize, 
                           rettype="xml", api_key=API_key) as handle:
            data = handle.read()
            article_data.append(data)
        
        time.sleep(10)

    print("Data fetching complete.")
    return article_data

In [ ]:
Entrez.email = input("Enter emailadres:")
query = build_query()
webenv, query_key, count, API_key = history(query)
articles = fetch_data(webenv, query_key, count, API_key)

In [5]:
"""
    To retrieve more than 10,000 UIDs from databases other than PubMed, 
    submit multiple esearch requests while incrementing the value of retstart (see Application 3). 
    For PubMed, ESearch can only retrieve the first 10,000 records matching the query. 
    To obtain more than 10,000 PubMed records, consider using <EDirect> that contains additional logic
    to batch PubMed search results automatically so that an arbitrary number can be retrieved.
    For details see https://www.ncbi.nlm.nih.gov/books/NBK25499/
"""

'\n    To retrieve more than 10,000 UIDs from databases other than PubMed, \n    submit multiple esearch requests while incrementing the value of retstart (see Application 3). \n    For PubMed, ESearch can only retrieve the first 10,000 records matching the query. \n    To obtain more than 10,000 PubMed records, consider using <EDirect> that contains additional logic\n    to batch PubMed search results automatically so that an arbitrary number can be retrieved.\n    For details see https://www.ncbi.nlm.nih.gov/books/NBK25499/\n'